In [114]:
import pandas as pd

In [115]:
df=pd.read_csv("uber_data.csv")

In [116]:
df["tpep_pickup_datetime"]=pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"]=pd.to_datetime(df["tpep_dropoff_datetime"])

In [117]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [118]:
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)

datetime_dim["Datetime_id"]=datetime_dim.index

datetime_dim["tpep_pickup_datetime"]=datetime_dim["tpep_pickup_datetime"]
datetime_dim["pick_hour"]=datetime_dim["tpep_pickup_datetime"].dt.hour
datetime_dim["pick_day"]=datetime_dim["tpep_pickup_datetime"].dt.day
datetime_dim["pick_month"]=datetime_dim["tpep_pickup_datetime"].dt.month
datetime_dim["pick_year"]=datetime_dim["tpep_pickup_datetime"].dt.year
datetime_dim["pick_weekday"]=datetime_dim["tpep_pickup_datetime"].dt.weekday

datetime_dim["tpep_dropoff_datetime"]=datetime_dim["tpep_dropoff_datetime"]
datetime_dim["drop_hour"]=datetime_dim["tpep_pickup_datetime"].dt.hour
datetime_dim["drop_day"]=datetime_dim["tpep_pickup_datetime"].dt.day
datetime_dim["drop_month"]=datetime_dim["tpep_pickup_datetime"].dt.month
datetime_dim["drop_year"]=datetime_dim["tpep_pickup_datetime"].dt.year
datetime_dim["drop_weekday"]=datetime_dim["tpep_pickup_datetime"].dt.weekday

In [119]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]


In [120]:
trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]

In [121]:
pickup_location_dim=df[["pickup_longitude","pickup_latitude"]].reset_index(drop=True)
pickup_location_dim["pickup_location_id"]=pickup_location_dim.index

In [122]:
dropoff_location_dim=df[["dropoff_longitude","dropoff_latitude"]].reset_index(drop=True)
dropoff_location_dim["dropoff_location_id"]=dropoff_location_dim.index

In [123]:
rate_code_mapping = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride"
}

rate_code_dim = df["RatecodeID"].drop_duplicates().reset_index(drop=True)
rate_code_dim = pd.DataFrame({'rate_code_id': rate_code_dim.index, 'RatecodeID': rate_code_dim})
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_mapping)
rate_code_dim = rate_code_dim[['rate_code_id', 'RatecodeID', 'rate_code_name']]


In [124]:
payment_type_mapping = {
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip"
}

payment_type_dim = df["payment_type"].drop_duplicates().reset_index(drop=True)
payment_type_dim = pd.DataFrame({'payment_type_id': payment_type_dim.index, 'payment_type': payment_type_dim})
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type_id'].map(payment_type_mapping)

payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]


In [125]:
payment_type_dim

,payment_type_id,payment_type,payment_type_name
0,0,1,NaN
1,1,2,Credit card
2,2,3,Cash
3,3,4,No charge


In [129]:
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id') \
             .merge(datetime_dim, left_on='trip_id', right_on='Datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id', 'VendorID', 'Datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]


In [130]:
fact_table

,trip_id,VendorID,Datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,1,1,1,1,1,N,1,1,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2,2,2,2,2,N,2,2,2,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,3,2,3,3,3,3,N,3,3,3,31.5,0.0,0.5,3.78,5.54,0.3,41.62
